# Free energy and phase equilibria of binary systems

_Authors: Enze Chen (University of California, Berkeley)_

![Eutectic phase diagram](https://raw.githubusercontent.com/enze-chen/learning_modules/master/fig/eutectic_PD.png)

This is an interactive notebook for you to see how changing the temperature affects the Gibbs free energy curves, $\Delta G \left( x_{\mathrm{B}} \right)$, of different phases in a binary system.
Moreover, it uses the common tangent construction to calculate a corresponding phase diagram and visually depicts the correspondence, which is admittedly something that was lost on me until I started the PhD. 😬
Hope you find this helpful!

## How to use this notebook

If you are viewing this notebook on Google Colaboratory, then everything is already set up for you (hooray🎉).
To use the widget, **run all the cells** (e.g. `Runtime > Run all`) and then adjust the sliders at the bottom.
I **strongly recommend** just running the whole notebook and experimenting with the inputs *before* reading the code in great detail.

If you want to **save a copy** of this notebook for yourself, go to "File > Save a copy in Drive" and you will find it in your Google Drive account under "My Drive > Colab Notebooks."

----------------------------------------

## Learning goals

By the end of this notebook, you should be able to:
- Describe how temperature and composition change the phase of a binary system through a $\Delta G$ argument.
- Construct model binary phase diagrams using the common tangent construction to Gibbs free energy curves.
- Describe the difference between ideal and regular solution models and how this manifests in different phase behaviors.

The hidden cells below provide the relevant background theory and the Python implementation of the widget. 
Reading is optional, but **the cells must be run** for the widget to work.

### Review of background theory

We know from thermodynamics that in general,

$$ \Delta G = \Delta H - T \Delta S$$

where $\Delta H$ is the enthalpy, $T$ is the temperature, and $\Delta S$ is the entropy. 
Hopefully this is very familiar to you!

In the regular solution model, we have an interaction term in the enthalpy given by the $\beta$ parameter, where

$$ \Delta H_{\text{mix}} = \beta x_{\mathrm{B}} \left( 1 - x_{\mathrm{B}} \right) $$

and $x_{\mathrm{B}}$ is the atomic fraction of species $\text{B}$ in the $\text{A}$--$\text{B}$ solutions. 
For ideal solutions, we can simply set $\beta = 0$. 
The entropy of mixing is given by

$$ \Delta S_{\text{mix}} = -R \left[ x_{\mathrm{B}} \ln x_{\mathrm{B}} + \left( 1 - x_{\mathrm{B}} \right) \ln \left( 1 - x_{\mathrm{B}} \right) \right] $$

Furthermore, we'll assume for this demo that the free energies of the pure solid phases are $\Delta G_{S}^{\alpha} = \Delta G_{S}^{\beta} = 0$ (reference).

------------------------------------- 

### Python implementation 

Our widget will call `plot_all()` each time we interact with it. 
This function calculates the Gibbs free energies as a function of concentration and plots $\Delta G$ and the phase diagram.
I've tried my best to keep the code simple and illustrative. 
Please see the comments in the code for more information.

In [1]:
# General libraries
import warnings
warnings.filterwarnings('ignore')  # please do as I say, not as I do

# Scientific computing libraries
import numpy as np
from scipy.misc import derivative
import matplotlib.pyplot as plt
%matplotlib inline

# Interactivity libraries
from ipywidgets import IntSlider, RadioButtons, Checkbox, \
                       Layout, HBox, GridspecLayout, interactive_output

#### Base functions

Here are the three functions for $\Delta G_{S}$, $\Delta G_{L}^{\mathrm{reg}}$, and $\Delta G_{L}^{\mathrm{ideal}}$.

In [2]:
kB = 8.314

def curve_s(x, T, beta=0):
    """This function plots the Gibbs free energy curve for the solid solution.
    
    Args:
        x (numpy.ndarray): An array of atomic fractions of B.
        T (float): The temperature in Kelvin.
        beta (float): The interaction parameter in J/mol.
        
    Returns:
        An array of Gibbs free energy values in kJ/mol.    
    """
    S_mix = -kB * (x * np.log(x) + (1 - x) * np.log(1 - x))
    H_mix = beta * x * (1 - x)
    G_s = -T * S_mix + H_mix
    return G_s / 1000


def curve_l_reg(x, T, beta=0):
    """This function plots the Gibbs free energy curve for the liquid
       using a regular solution model and assuming close melting points.
    
    Args:
        x (numpy.ndarray): An array of atomic fractions of B.
        T (float): The temperature in Kelvin.
        beta (float): The interaction parameter in J/mol.
        
    Returns:
        An array of Gibbs free energy values in kJ/mol.    
    """
    S_A, S_B = (7.773, 8.167)
    T_A, T_B = (1100, 1200)   # melting points
    G_A = S_A * (T_A - T)
    G_B = S_B * (T_B - T)
    S_mix = -kB * (x * np.log(x) + (1 - x) * np.log(1 - x))
    H_mix = beta * x * (1 - x)
    G_l = x * G_B + (1 - x) * G_A - T * S_mix + H_mix
    return G_l / 1000


def curve_l_ideal(x, T):
    """This function plots the Gibbs free energy curve for the liquid
       using an ideal solution model and assuming disparate melting points.
    
    Args:
        x (numpy.ndarray): An array of atomic fractions of B.
        T (float): The temperature in Kelvin.
        beta (float): The interaction parameter in J/mol.
        
    Returns:
        An array of Gibbs free energy values in kJ/mol.    
    """
    S_A, S_B = (16, 14.7)
    T_A, T_B = (1163, 578)   # melting points
    G_A = S_A * (T_A - T)
    G_B = S_B * (T_B - T)
    S_mix = -kB * (x * np.log(x) + (1 - x) * np.log(1 - x))
    G_l = x * G_B + (1 - x) * G_A - T * S_mix
    return G_l / 1000

#### Common tangents

This function computes the common tangent to two curves.
There are many ways to do this, perhaps most straightforwardly using something like [SciPy](https://scipy.org/) to solve a [non-linear system of equations](https://www.tf.uni-kiel.de/matwis/amat/td_kin_ii/kap_1/backbone/r_se15.html) and most cleverly using a [Legendre transform](https://en.wikipedia.org/wiki/Legendre_transformation#Thermodynamics).
My way is quite hacky but relatively straightforward: I will make a guess for the location of the common tangent and iteratively adjust the location until I approach the real answer.
I'm not sure if this is the fastest way, but it is sufficient.

In [3]:
def common_tangent(x, y1, y2, fn, T, beta=0):
    """This function calculates the common tangent(s) between two curves.
       It can likely be improved to incorporate known physics.
    
    Args:
        x (numpy.ndarray): An array of atomic fractions of B.
        y1 (numpy.ndarray): y values for curve 1.
        y2 (numpy.ndarray): y values for curve 2.
        fn (function): The function that we take the derivative of.
        T (float): The temperature in Kelvin.
        beta (float): The interaction parameter in J/mol.
        
    Returns:
        line (numpy.ndarray): y values for the common tangent.
        idmin (int): Index of the x-coordinate of the first tangent point.
        idmax (int): Index of the x-coordinate of the second tangent point.
    """
    # Compute a derivative
    dy = derivative(func=fn, x0=x, dx=1e-3, args=(T, beta))

    # Make an initial guess at the minimum of curve 1
    idmin, idmax = (0, len(x))
    idx = np.argmin(y1)
    xp, yp, dyp = x[idx], y1[idx], dy[idx]

    # Construct the tangent line and count intersections with curve 2
    thresh = 1
    line = dyp * x + (yp - dyp * xp)
    diff = np.diff(np.sign(y2 - line))   # check for sign changes!
    nnz = np.count_nonzero(diff)

    # CASE 1: y1 and y2 are the same curve. Find the miscibility gap.
    if np.linalg.norm(y1 - y2) < 1e-4:
        idmin = np.argmin(y1[:int(n/2)])
        idmax = np.argmin(y1[int(n/2):]) + int(n/2)
    
    # CASE 2: If the tangent intersects y2, shift tangent point to the left
    elif nnz >= thresh:
        while nnz >= thresh:
            idx -= 1
            # try-except to avoid an out-of-bounds error 
            try:
                xp, yp, dyp = x[idx], y1[idx], dy[idx]
                line = dyp * x + (yp - dyp * xp)
                diff = np.diff(np.sign(y2 - line))
                nnz = np.count_nonzero(diff)
            except:
                break
            if diff.any():
                # Assign left and right indices of the tangent points
                # We have to do it each time because once we miss, we can't go back
                idmax = np.nonzero(diff)[0][0]
        idmin = idx

    # CASE 3: If the tangent misses y2, shift tangent point to the right
    elif nnz < thresh:
        while nnz < thresh:
            idx += 1
            # try-except to avoid an out-of-bounds error 
            try:
                xp, yp, dyp = x[idx], y1[idx], dy[idx]
                line = dyp * x + (yp - dyp * xp)
                diff = np.diff(np.sign(y2 - line))
                nnz = np.count_nonzero(diff)
            except:
                break
        # Assign left and right indices of the tangent points
        idmin = idx
        idmax = np.nonzero(diff)[0][0]
    
    # Return a tuple
    return (line, idmin, idmax)

#### Calculate the phase boundaries in advance

This does not have to be calculated in advance _per se_, especially if students want to be able to change parameters like $\beta_s$ and $T_m$, but pre-computing the location of the phase boundaries makes the widget animation respond much faster when drawing the phase diagrams.

In [4]:
# Enumerate the temperatures we want to calculate phase boundaries at
Ts = [500, 550, 579, 600, 700, 800, 892, 900, 950, 1000, 1050, 1090, 1100, 1150, 1190]
beta_s = 20000
beta_l = 10000

# Create empty arrays to track the x-coordinates of the boundaries
liquidus_r = []
solvus_r1 = []
solvus_r2 = []
liquidus_i = []
solidus_i = []

# Create x-axis of concentrations
n = int(1e4)
xmin, xmax = (0.001, 0.999)
xx = np.linspace(xmin, xmax, n)

plot_sl = False   # controls how many tangent points are processed
for T in Ts:
    # Regular solution model
    y_s = curve_s(xx, T, beta_s)
    y_l = curve_l_reg(xx, T, beta_l)

    # First check solid-solid boundaries
    line, idmin, idmax = common_tangent(xx, y_s, y_s, curve_s, T, beta_s)
    lmin = np.argmin(y_l)
    if line[lmin] <= y_l[lmin]:
        ids = np.absolute([idmin, idmax])
    # There are solid-liquid common tangent(s)
    else:
        line1, idmin1, idmax1 = common_tangent(xx, y_s, y_l, curve_s, T, beta_s)
        line2, idmin2, idmax2 = common_tangent(xx, y_l, y_s, curve_l_reg, T, beta_l)
        ids = np.absolute([idmin1, idmax1, idmin2, idmax2])
        plot_sl = True
    
    if (ids < n).all():
        if plot_sl:
            if ids[0] < ids[1]:
                solvus_r1.append((xx[ids[0]], T))
                liquidus_r.append((xx[ids[1]], T))
            if ids[2] < ids[3]:
                liquidus_r.append((xx[ids[2]], T))
                solvus_r2.append((xx[ids[3]], T))
        else:
            if ids[0] < ids[1]:
                solvus_r1.append((xx[ids[0]], T))
                solvus_r2.append((xx[ids[1]], T))
                
    # Ideal solution model
    y_s = curve_s(xx, T) 
    y_l = curve_l_ideal(xx, T) 
    line, idmin, idmax = common_tangent(xx, y_s, y_l, curve_s, T)
    ids = np.absolute([idmin, idmax])
    if (ids < n).all() and ids[0] < ids[1]:
        liquidus_i.append((xx[idmax], T))
        solidus_i.append((xx[idmin], T))

# Sort the lists for plotting purposes
liquidus_r.sort(key=lambda x: x[0], reverse=False)
solvus_r1.sort(key=lambda x: x[1], reverse=False)
solvus_r2.sort(key=lambda x: x[1], reverse=False)

#### Plotting functions

With the helper functions in hand, the following functions plot the various curves based on user inputs.
The first block of code is for the regular solution model and eutectic phase diagram and the second block of code is for the ideal solution model and lens phase diagram.
The final block processes the logic for the widget.

In [5]:
def plot_Gx_reg(ax1, ymin, T=1000, beta_s=0, beta_l=0, CT=True):
    """This function calculates and plots Gibbs free energy curves.
       The mechanics are very similar to the previous block.

    Args:
        ax1 (Axes): Axes object from matplotlib.
        ymin (float): A minimum value for vertical guides.
        T (float): The temperature in Kelvin.
        beta_s (float): The interaction parameter for solids in J/mol.
        beta_l (float): The interaction parameter for liquids in J/mol.
        CT (bool): Plot common tangent.

    Returns:
        A list of x-coordinates of tangent points.
    """
    # For the given temperature, calculate the curves and common tangent
    plot_sl_tangent = False
    y_s = curve_s(xx, T, beta_s)
    y_l = curve_l_reg(xx, T, beta_l)

    if CT:
        # First check solid-solid common tangent
        line, idmin, idmax = common_tangent(xx, y_s, y_s, curve_s, T, beta_s)
        lmin = np.argmin(y_l)
        if line[lmin] <= y_l[lmin]:
            ids = np.absolute([idmin, idmax])
        # There are solid-liquid common tangent(s)
        else:
            line1, idmin1, idmax1 = common_tangent(xx, y_s, y_l, curve_s, T, beta_s)
            line2, idmin2, idmax2 = common_tangent(xx, y_l, y_s, curve_l_reg, T, beta_l)
            ids = np.absolute([idmin1, idmax1, idmin2, idmax2])
            plot_sl_tangent = True
    else:
        ids = np.full((2,), np.inf)
    xpts = []
    
    # Mostly plot settings for visual appeal
    ax1.plot(xx, y_l, c='C1', label='liquid')
    ax1.plot(xx, y_s, c='C0', label='solid')
    if (ids < n).all() and CT:
        if plot_sl_tangent:
            ax1.annotate(text=r'$L$', xy=(0.4, -5.5))
            if ids[0] < ids[1]:
                ax1.plot(xx[idmin1:idmax1], line1[idmin1:idmax1], c='k', ls='-.')
                xlow, xhigh = (xx[idmin1], xx[idmax1])
                xpts.extend([xlow, xhigh])
                ax1.vlines(x=[xlow, xhigh], ymin=ymin, \
                          ymax=[line1[idmin1], line1[idmax1]], color='gray', linestyles='dotted', linewidth=2)
                ax1.annotate(text=r'$\alpha$', xy=(xlow/2 - 0.02, -3.8))
                ax1.annotate(text=r'$\alpha+L$', xy=((xlow + xhigh)/2 - 0.04, -4.8))
            if ids[2] < ids[3]:
                ax1.plot(xx[idmin2:idmax2], line2[idmin2:idmax2], c='k', ls='-.')
                xlow, xhigh = (xx[idmin2], xx[idmax2])
                xpts.extend([xlow, xhigh])
                ax1.vlines(x=[xlow, xhigh], ymin=ymin, \
                          ymax=[line2[idmin2], line2[idmax2]], color='gray', linestyles='dotted', linewidth=2)
                ax1.annotate(text=r'$L+\beta$', xy=((xlow + xhigh)/2 - 0.04, -4.8))
                ax1.annotate(text=r'$\beta$', xy=((xhigh + 1)/2 - 0.02, -3.8))            
        elif ids[0] < ids[1]:
            ax1.plot(xx[idmin:idmax], line[idmin:idmax], c='k', ls='-.')
            xlow, xhigh = (xx[idmin], xx[idmax])
            xpts.extend([xlow, xhigh])
            ax1.vlines(x=[xlow, xhigh], ymin=ymin, \
                      ymax=[line[idmin], line[idmax]], color='gray', linestyles='dotted', linewidth=2)
            ax1.annotate(text=r'$\alpha$', xy=(xlow/2 - 0.02, -3.8))
            ax1.annotate(text=r'$\alpha+\beta$', xy=((xlow + xhigh)/2 - 0.05, -4.8))
            ax1.annotate(text=r'$\beta$', xy=((xhigh + 1)/2 - 0.02, -3.8))
    ax1.legend(loc='upper right', framealpha=0.6)
    return xpts


def plot_pd_reg(ax2, T, xpts):
    """This function calculates and plots a lens phase diagram based on 
       precomputed phase boundaries.

    Args:
        ax2 (Axes): Axes object from matplotlib.
        T (float): The temperature in Kelvin.
        xpts (list): A list of x-coordinates of tangent points

    Returns:
        None, but a pyplot is displayed.
    """
    ax2.plot(*zip(*(max(solvus_r1), min(solvus_r2))), 'k', lw=1.5, alpha=0.4)
    ax2.plot(*zip(*solvus_r1), '-', c='C0', label='solvus1')
    ax2.plot(*zip(*liquidus_r), '-', c='C1', label='liquidus')
    ax2.plot(*zip(*solvus_r2), '-', c='C0', label='solvus2')
    ax2.vlines(x=xpts, ymin=T, ymax=1300, color='gray', linestyles='dotted', linewidth=2)
    ax2.hlines(y=T, xmin=0, xmax=1, color='C3', linestyles='dotted', linewidth=2)
    ax2.annotate(text=r'$\alpha+\beta$', xy=(0.41, 700))
    ax2.annotate(text=r'$\alpha$', xy=(0.02, 850))
    ax2.annotate(text=r'$\beta$', xy=(0.94, 850))
    ax2.annotate(text=r'$\alpha+L$', xy=(0.1, 920))
    ax2.annotate(text=r'$L$', xy=(0.43, 1050))
    ax2.annotate(text=r'$L+\beta$', xy=(0.66, 920))

In [6]:
def plot_Gx_ideal(ax1, ymin, T=800, CT=True):
    """This function calculates and plots Gibbs free energy curves.
       The mechanics are very similar to the previous block.

    Args:
        ax1 (Axes): Axes object from matplotlib.
        ymin (float): A minimum value for vertical guides.
        T (float): The temperature in Kelvin.
        CT (bool): Plot common tangent.

    Returns:
        A list of x-coordinates of tangent points.
    """
    # For the given temperature, calculate the curves and common tangent
    y_s = curve_s(xx, T)
    y_l = curve_l_ideal(xx, T)
    if CT:
        line, idmin, idmax = common_tangent(xx, y_s, y_l, curve_s, T)
    else:
        idmin = np.inf
    xpts = []

    # Mostly plot settings for visual appeal
    ax1.plot(xx, y_l, c='C1', label='liquid')
    ax1.plot(xx, y_s, c='C0', label='solid')
    if abs(idmin) < n and abs(idmax) < n and CT:
        xlo, xhi = xx[idmin], xx[idmax]
        ax1.plot(xx[idmin:idmax], line[idmin:idmax], c='k', ls='-.')
        ax1.vlines(x=[xlo, xhi], ymin=ymin, \
                  ymax=[line[idmin], line[idmax]], color='gray', linestyles='dotted', linewidth=2)
        xpts.extend([xlo, xhi])
        ax1.annotate(text=r'$S$', xy=(xlo/2 - 0.02, -12))
        if xlo < xhi:
            ax1.annotate(text=r'$S+L$', xy=((xlo+xhi)/2 - 0.05, -13))
            ax1.annotate(text=r'$L$', xy=((xhi+1)/2 - 0.02, -12))
    ax1.legend(loc='upper right', framealpha=0.6)
    return xpts


def plot_pd_ideal(ax2, T, xpts):
    """This function calculates and plots a lens phase diagram based on 
       precomputed phase boundaries.

    Args:
        ax2 (Axes): Axes object from matplotlib.
        T (float): The temperature in Kelvin.
        xpts (list): A list of x-coordinates of tangent points

    Returns:
        None, but a pyplot is displayed.
    """
    ax2.plot(*zip(*liquidus_i), '-', c='C1', label='liquidus')
    ax2.plot(*zip(*solidus_i), '-', c='C0', label='solidus')
    ax2.vlines(x=xpts, ymin=T, ymax=2500, color='gray', linestyles='dotted', linewidth=2)
    ax2.hlines(y=T, xmin=0, xmax=1, color='C3', linestyles='dotted', linewidth=2)
    ax2.annotate(text=r'$S$', xy=(0.4, 600))
    ax2.annotate(text=r'$S+L$', xy=(0.65, 730))
    ax2.annotate(text=r'$L$', xy=(0.7, 1050))

In [7]:
def plot_all(T, S, G, P, CT):
    plt.rcParams.update({'figure.figsize':(4.5,5.5), 'font.size':18, 'mathtext.fontset':'cm',
                         'lines.linewidth':3, 'axes.linewidth':1.5, 
                         'xtick.direction':'in', 'ytick.direction':'in', 
                         'xtick.major.size':5, 'xtick.major.width':1.5, 
                         'ytick.major.size':5, 'ytick.major.width':1.5, 
                         'xtick.top':True, 'ytick.right':True,
                         'axes.spines.right':True, 'axes.spines.top':True,})
    fig, (ax1, ax2) = plt.subplots(nrows=2, sharex=True, layout='tight')
    ax1.set_title(f'Phase equilibria at $T = {T}$ K', pad=10)
    ax1.set_ylabel(r'$\Delta G$ (kJ mol$^{-1}$)')
    ax2.set_xlabel(r'$x_{\mathrm{B}}$')
    ax2.set_ylabel(r'$T$ (K)', color='C3')
    ax2.tick_params(axis='y', colors='C3')
    ax2.set_xlim(0, 1)
    ax2.set_ylim(501, 1299)
    
    if S == 'Regular':
        if G:
            ymin, ymax = (-6.5, 5)
            ax1.set_ylim(ymin, ymax)
            xpts = plot_Gx_reg(ax1, ymin, T, beta_s, beta_l, CT)
            if P:
                plot_pd_reg(ax2, T, xpts)
        elif P:
            plot_pd_reg(ax2, T, None)
    elif S == 'Ideal':
        if G:
            ymin, ymax = (-15, 8)
            ax1.set_ylim(ymin, ymax)
            xpts = plot_Gx_ideal(ax1, ymin, T, CT)
            if P:
                plot_pd_ideal(ax2, T, xpts)
        elif P:
            plot_pd_ideal(ax2, T, None)
    plt.show()


T_widget = IntSlider(value=850, min=550, max=1250, step=50, continuous_update=False, 
                     description='Temperature (K)', readout_format='d', 
                     style={'description_width':'100px', 'handle_color':'#d62728'}, 
                     layout=Layout(width='400px', height='30px'))
S_widget = RadioButtons(value='Ideal', options=['Ideal', 'Regular'], description='Solid solution:',
                        style={'description_width':'100px'}, continuous_update=False,
                        layout=Layout(width='250px', height='20px'))
G_widget = Checkbox(value=True, description='Show ΔG curves?', indent=False)
CT_widget = Checkbox(value=False, description='Plot common tangent?', indent=False)
P_widget = Checkbox(value=False, description='Show phase diagram?', indent=False)
h = HBox([G_widget, CT_widget, P_widget])

g = GridspecLayout(n_rows=2, n_columns=2, height='85px', width='750px')
g[0, 0] = T_widget
g[0, 1] = S_widget
g[1, :] = h

out = interactive_output(plot_all, {'T':T_widget, 'S':S_widget, 'G':G_widget, 'CT':CT_widget, 'P':P_widget});

## Interactive widget

Slide the temperature bar (_please wait for the computation after releasing_) and see how the free energy curves change.
Toggle the phase diagram button to reveal the phase diagram, and see how the common tangents in the top plot correspond to phase boundaries in the lower plot.
**Please make sure you've ran the hidden cells above** before running the next cell (easiest to just click "Runtime > Run all" in the menu bar). 

In [8]:
display(g, out)

GridspecLayout(children=(IntSlider(value=850, continuous_update=False, description='Temperature (K)', layout=L…

Output()

----------

## Conclusion

I hope you found this notebook helpful in learning more about free energy curves and their correspondence to phase diagrams.
Please don't hesitate to reach out if you have any questions or ideas to contribute.